<a href="https://colab.research.google.com/github/step-cheng/CIFAR10-pytorch/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torch>=2.0.1
# !pip install torchvision
# !pip install -Uqq ipdb

In [1]:
!git clone https://github.com/step-cheng/CIFAR10-pytorch.git

Cloning into 'CIFAR10-pytorch'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 67 (delta 30), reused 40 (delta 15), pack-reused 0
Receiving objects: 100% (67/67), 24.99 KiB | 1.14 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
# %cd CIFAR10-pytorch
# !git pull
# %cd ..

In [ ]:
# Turn on Google Colab Debugger
# %pdb on
# %pdb off

In [2]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
# import ipdb

In [3]:
import sys
sys.path.append('/content/CIFAR10-pytorch')
from models import *

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:04<00:00, 128MB/s]


In [5]:
# 50000 train images
train_data = datasets.CIFAR10(root='./', train=True, download=True, transform=transforms.ToTensor(),
                              target_transform=transforms.Lambda(lambda y : torch.zeros(10,dtype=torch.float).scatter(dim=0,
                                                                                                                       index=torch.tensor(y),
                                                                                                                       value=1)))
# DataLoader wraps an iterable over dataset for automatic batching, sampling, and dataloading
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

# 10000 test images
test_data = datasets.CIFAR10(root='./', train=False, download=True, transform=transforms.ToTensor(),
                             target_transform=transforms.Lambda(lambda y : torch.zeros(10,dtype=torch.float).scatter(dim=0,
                                                                                                                       index=torch.tensor(y),
                                                                                                                       value=1)))
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

classes = {
    0 : 'plane',    1 : 'car',    2 : 'bird',   3 : 'cat',    4 : 'deer',
    5 : 'dog',      6 : 'frog',   7 : 'horse',  8 : 'ship',   9 : 'truck'
}

100%|██████████| 170498071/170498071 [00:01<00:00, 100300294.83it/s]


Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [6]:
# img, label = training_data[1]
train_iter = iter(train_loader)
# img, label = next(train_iter)
# print(img.shape)
# print(label.shape)

# shows image tensor
def show(tens, label):
  # make_grid makes a grid of images from a batch of tensors BxCxHxW
  label = torch.argmax(label,dim=1)
  print(label)
  img = torchvision.utils.make_grid(tens) # flattens B dimension to make a grid
  np_img = img.numpy()
  plt.imshow(np.transpose(np_img, (1,2,0)))     # seems each image is spaced by 2 pixels, channels goes last when plotting
  # title = classes[label]
  title = ' '.join([classes[t.item()] for t in label])
  plt.title(title)

# show(img, label)


In [ ]:
# # Demonstration of loss.backward() and optimizer.step()

# print(model.conv_pool_stack[0].weight[0,0])
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

# input = torch.randn((4,3,32,32),requires_grad=False)
# target = torch.randint(0,10,size=(4,))
# logits = model(input)
# loss = loss_fn(logits,target)
# print(loss)
# loss.backward()
# print(model.conv_pool_stack[0].weight[0,0])
# print(model.conv_pool_stack[0].weight.grad[0,0])
# optimizer.step()
# print(model.conv_pool_stack[0].weight[0,0])
# print(model.conv_pool_stack[0].weight.grad[0,0])
# optimizer.zero_grad()

In [7]:
def preprocess(x):
  # find std and mean per channel
  x_stds = torch.std(x, dim=(0,2,3))
  x_means = torch.sum(x, dim=(0,2,3)) / (x.shape[0]*x.shape[2]*x.shape[3])

  for channel in range(3):
    x[:,channel,:,:] -= x_means[channel]
    x[:,channel,:,:] /= x_stds[channel]
  return x


In [8]:
def accuracy(pred,y):
  guesses = torch.argmax(pred,dim=1)
  matches = sum(guesses==y).item()
  return matches/torch.numel(y), matches


In [9]:
def train(dataloader, model, device, loss_fn, optimizer):
  # sets the module to training mode, eval does the opposite
  model.train()

  accs = []
  losses = []
  matches = 0

  for batch, (X, y) in enumerate(dataloader):
    imgs, labels = X.to(device), y.to(device)

    imgs = preprocess(imgs)
    pred = model(imgs)
    loss = loss_fn(pred, torch.argmax(labels,dim=1))
    losses.append(loss.item())
    acc, m = accuracy(pred,torch.argmax(labels,dim=1))
    accs.append(acc)
    matches += m

    if batch % 50 == 0:
      print(f"Progress: {(batch+1) * len(X)}, current accuracy: {m/len(X)}")

    # Backpropagation
    loss.backward()     # backpropagates prediction loss, deposits gradients of the loss w.r.t. for each parameter that has requires_grad=True
    # print(model.classifier[2].weight[:,0])
    # print(model.classifier[2].weight.grad[:,0])
    optimizer.step()    # updates the parameter values, "learns"
    optimizer.zero_grad()       # zeros the grads because it is a dynamic graph

  f, (ax1,ax2) = plt.subplots(1,2)
  ax1.plot(torch.arange(batch+1),losses)
  ax2.plot(torch.arange(batch+1),accs)
  ax1.set_title("Loss vs Epoch")
  ax1.set_ylabel("Loss")
  ax1.set_xlabel("Epoch")
  ax2.set_title("Accuracy vs Epoch")
  ax2.set_ylabel("Accuracy")
  ax2.set_xlabel("Epoch")

  print(f"Epoch Accuracy: {matches/50000}")



In [10]:
def test(dataloader, model, device):

  # set module to testing mode, train does the opposite
  model.eval()
  accs = []
  matches = 0

  # used to turn off gradient calculations
  with torch.no_grad():
    for batch, (X,y) in enumerate(dataloader):
      imgs, labels = X.to(device), y.to(device)

      imgs = preprocess(imgs)
      pred = model(imgs)
      acc, m = accuracy(pred,torch.argmax(labels,dim=1))
      matches += m

      if batch % 50 == 0:
        print(f"Progress: {(batch+1)*len(X)}, current accuracy: {m/len(X)}")
        accs.append(acc)

  print(f"Test Accuracy: {matches/10000}")

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Pytorch version: {torch.__version__}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available.")

No GPU available.


In [12]:
model = transfer.myVgg19.to(device)
# model.half() # If doing this, input type and weight type need to be the same

# set cross entropy loss and Adam optimizer functions
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3) # 5e-4 for mymodel

In [13]:
epochs = 5
start = datetime.now()
for e in range(epochs):
  train(train_loader, model, device, loss_fn, optimizer)
  test(test_loader,model,device)
elapse = datetime.now() - start
print("Training Time: ", elapse)


Progress: 64, current accuracy: 0.078125
Progress: 3264, current accuracy: 0.453125
Progress: 6464, current accuracy: 0.390625
Progress: 9664, current accuracy: 0.5
Progress: 12864, current accuracy: 0.53125
Progress: 16064, current accuracy: 0.53125
Progress: 19264, current accuracy: 0.453125
Progress: 22464, current accuracy: 0.5
Progress: 25664, current accuracy: 0.453125


KeyboardInterrupt: ignored

In [ ]:
# Show missed images and what the machine thought it was
# Visualize the conv filter?